### Importing Libraries

In [10]:
#import dependencies
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy
import re

#sentiment analysis
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

#keyword Extraction
from sklearn.feature_extraction.text import TfidfVectorizer


#visuals
import matplotlib.pyplot as plt
from wordcloud import WordCloud

### Load bank_reviews dataset 

In [45]:
df = pd.read_csv('bank_reviews.csv')
df.head()

,review,rating,date,bank,source
0,the app is proactive and a good connections.,5,2025-06-05,CBE,Google Play
1,I cannot send to cbebirr app. through this app.,3,2025-06-05,CBE,Google Play
2,good,4,2025-06-05,CBE,Google Play
3,not functional,1,2025-06-05,CBE,Google Play
4,everytime you uninstall the app you have to re...,1,2025-06-04,CBE,Google Play


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6670 entries, 0 to 6669
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  6669 non-null   object
 1   rating  6670 non-null   int64 
 2   date    6670 non-null   object
 3   bank    6670 non-null   object
 4   source  6670 non-null   object
dtypes: int64(1), object(4)
memory usage: 260.7+ KB


In [67]:
df['review']

0            the app is proactive and a good connections.
1         I cannot send to cbebirr app. through this app.
2                                                    good
3                                          not functional
4       everytime you uninstall the app you have to re...
                              ...                        
6665                                            Nice one.
6666                           በጣም ደስ የሚል ለውጥ አናመሰግናለን 🇪🇹
6667         Best application👍Thank you ! Abyssinia bank.
6668    Absolutely it's fantastic apps this New apps i...
6669           The best app next to Tele birr in ethiopia
Name: review, Length: 6670, dtype: object

### Preprocessing the Data¶
Steps:

1. Clean text: Remove punctuation, special characters, convert to lowercase.
2. Tokenize: Split text into words.
3. Remove stop words: Eliminate common words (e.g., "the," "and").
4. Lemmatize: Reduce words to base form (e.g., "running" → "run").




* As shown the reveiw column in our data contains Amharic sentences so I have to translate it first to English
### Use Google Translate API
### Step 1: Install googletrans
### pip install googletrans==4.0.0-rc1


### Step 2: Translate Function
### Step 3: Apply Translation to Amharic Reviews

In [68]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\smith\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\smith\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\smith\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\smith\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [75]:
from langdetect import detect
from transformers import pipeline
from deep_translator import GoogleTranslator
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import nltk

# Load sentiment pipeline (DistilBERT SST-2)
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_pipeline = pipeline("sentiment-analysis", model=model_name)

# Language detection function
def detect_lang(text):
    try:
        return detect(str(text)).lower()
    except:
        return 'unknown'

# Translate Amharic to English
def translate_amharic_to_english(text):
    try:
        if not isinstance(text, str) or text.strip() == "":
            return text
        return GoogleTranslator(source='am', target='en').translate(text)
    except Exception as e:
        print(f"Translation error: {e}")
        return text  # fallback to original text if translation fails

# Text preprocessing: tokenize, lowercase, remove stopwords, lemmatize
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    tokens = word_tokenize(text.lower())
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(filtered_tokens)

# Detect language column
df['lang'] = df['review'].apply(detect_lang)

# Translate Amharic reviews
df['translated_review'] = df.apply(
    lambda row: translate_amharic_to_english(row['review']) if row['lang'] == 'am' else row['review'],
    axis=1
)

# Preprocess all translated/English reviews
df['processed_review'] = df['translated_review'].apply(preprocess_text)



RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [42]:
def preprocess_text(text):
    if not isinstance(text, str):
        return ""  # or np.nan if you want to preserve NaNs
    tokens = word_tokenize(text.lower())
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(tokens)


# Apply preprocessing
df['processed_review'] = df['translated_review'].apply(preprocess_text)


In [44]:
df

,review,rating,date,bank,source,lang,translated_review,processed_review
0,the app is proactive and a good connections.,5,2025-06-05,CBE,Google Play,en,the app is proactive and a good connections.,app proactive good connection
1,I cannot send to cbebirr app. through this app.,3,2025-06-05,CBE,Google Play,en,I cannot send to cbebirr app. through this app.,send cbebirr app app
2,good,4,2025-06-05,CBE,Google Play,so,good,good
3,not functional,1,2025-06-05,CBE,Google Play,en,not functional,functional
4,everytime you uninstall the app you have to re...,1,2025-06-04,CBE,Google Play,en,everytime you uninstall the app you have to re...,everytime uninstall app reach physically oldy ...
...,...,...,...,...,...,...,...,...
6665,Nice one.,5,2024-01-14,BOA,Google Play,en,Nice one.,nice one
6666,በጣም ደስ የሚል ለውጥ አናመሰግናለን 🇪🇹,5,2024-01-14,BOA,Google Play,unknown,በጣም ደስ የሚል ለውጥ አናመሰግናለን 🇪🇹,በጣም ደስ የሚል ለውጥ አናመሰግናለን
6667,Best application👍Thank you ! Abyssinia bank.,5,2024-01-12,BOA,Google Play,en,Best application👍Thank you ! Abyssinia bank.,best abyssinia bank
6668,Absolutely it's fantastic apps this New apps i...,5,2024-01-11,BOA,Google Play,en,Absolutely it's fantastic apps this New apps i...,absolutely fantastic apps new apps fast good apps


In [15]:
nlp = spacy.load('en_core_web_sm')

def clean_text(text):
    text = re.sub(r'[^A-Za-z\s]', '', text.lower())
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])

df = pd.read_csv('bank_reviews.csv')
df['cleaned_text'] = df['review'].apply(clean_text)


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

### Sentiment Analysis (VADER)

In [ ]:
nltk.download('vader_lexicon')

In [3]:
sia = SentimentIntensityAnalyzer()
df['sentiment_score'] = df['review'].apply(lambda x: sia.polarity_scores(x)['compound'])
df['sentiment_label'] = df['sentiment_score'].apply(lambda x: 'positive' if x > 0.05 else 'negative' if x < -0.05 else 'neutral')

NameError: name 'SentimentIntensityAnalyzer' is not defined

### Aggregate Sentiment by Bank and Rating

In [ ]:
sentiment_summary = df.groupby(['bank_name', 'rating'])['sentiment_score'].mean().reset_index()

### Keyword Extraction (TF-IDF)

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1,2), max_features=100)
X = tfidf.fit_transform(df['cleaned_text'])
keywords_df = pd.DataFrame(X.toarray(), columns=tfidf.get_feature_names_out())


### Theme Clustering (Manual or Rule-Based)

In [ ]:
themes = {
    "Access Issues": ["login", "access", "reset", "otp"],
    "Transaction Performance": ["slow", "transfer", "delay", "fail"],
    "UI/UX": ["interface", "navigation", "easy", "design"],
    "Customer Support": ["support", "help", "response", "contact"]
}

def assign_theme(text):
    assigned = []
    for theme, keywords in themes.items():
        if any(kw in text for kw in keywords):
            assigned.append(theme)
    return ', '.join(assigned)

df['themes'] = df['cleaned_text'].apply(assign_theme)


### Save Output CSV

In [ ]:
df.to_csv('bank_reviews_with_sentiment_themes.csv', index=False)
